In [37]:
# Librerias necesarias
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import parametros as p
import pickle

In [2]:
# Cargar los datos
df = pd.read_excel("../2. data/0 TimeLog (original, no modificar).xlsx")

In [38]:
# Nuevos datos con requerimientos de las llegadas
with open('../2. data/llegadas.pkl', 'rb') as file:
    llegadas_n = pickle.load(file)

In [48]:
df.head(20)

,ID,MS_GRD,UBICACIÓN,TI,TF,HOSPITAL,UNIDAD
0,1,5,WL_WL,12,384,WL,WL
1,1,5,Hospital_1_GA,384,468,Hospital_1,GA
2,1,5,Hospital_1_ICU,468,552,Hospital_1,ICU
3,1,5,Hospital_1_SDU_WARD,576,768,Hospital_1,SDU_WARD
4,2,5,WL_WL,12,1224,WL,WL
5,2,5,PS_PS,1224,1224,PS,PS
6,3,6,WL_WL,12,12,WL,WL
7,3,6,Hospital_1_GA,12,12,Hospital_1,GA
8,3,6,Hospital_1_OR,12,24,Hospital_1,OR
9,3,6,Hospital_1_ICU,24,96,Hospital_1,ICU


In [53]:
print(len(df[(df["HOSPITAL"] == "WL") & (df["TI"] == 12)]))

cantidad = 0
for key, values in llegadas_n["WL"][1].items():
    cantidad += values
print(cantidad)


12
16


In [129]:
# Copio para no modificar el original
tl = df.copy()

In [ ]:
# Agrego LOS a cada fila y reordeno
tl["LOS"] = (tl["TF"] - tl["TI"])
tl = tl[["ID", "MS_GRD", "UBICACIÓN", "TI", "TF", "LOS", "HOSPITAL", "UNIDAD"]]

# Lista para guardar filas nuevas
new_rows = []

# Recorrer filas consecutivas
for i in range(len(tl) - 1):
    row_current = tl.iloc[i]
    row_next = tl.iloc[i + 1]

    # Verificar condiciones
    same_id = row_current['ID'] == row_next['ID']
    same_ms = row_current['MS_GRD'] == row_next['MS_GRD']
    time_gap = row_current['TF'] < row_next['TI']
    same_hospital = row_current['HOSPITAL'] == row_next['HOSPITAL']

    if same_id and same_ms and time_gap:
        # Crear fila intermedia
        new_row = {
            'ID': row_current['ID'],
            'MS_GRD': row_current['MS_GRD'],
            'UBICACIÓN': f"{row_current['UBICACIÓN']} -> {row_next['UBICACIÓN']}",
            'TI': row_current['TF'],
            'TF': row_next['TI'],
            'LOS': row_next['TI'] - row_current['TF'],
            'HOSPITAL': row_current['HOSPITAL'],
            'UNIDAD': row_current['UNIDAD']
        }
        new_rows.append((i + 1, new_row))  # guardar con índice de inserción
    
    if same_id and same_ms and not time_gap and not same_hospital:
        # Crear fila intermedia
        new_row = {
            'ID': row_current['ID'],
            'MS_GRD': row_current['MS_GRD'],
            'UBICACIÓN': f"{row_current['UBICACIÓN']} -> {row_next['UBICACIÓN']}",
            'TI': row_current['TF'],
            'TF': row_next['TI'],
            'LOS': row_next['TI'] - row_current['TF'],
            'HOSPITAL': row_current['HOSPITAL'],
            'UNIDAD': "En movimiento"
        }
        new_rows.append((i + 1, new_row))  # guardar con índice de inserción

# Lista para guardar todas las filas nuevas (dicts)
new_rows_list = [fila[1] for fila in new_rows]

# Convertir todo a un DataFrame una sola vez
new_rows_df = pd.DataFrame(new_rows_list)

# Concatenar en una sola pasada
tl = pd.concat([tl, new_rows_df], ignore_index=True)

# Reordenar filas para meter filas entremedio
tl = tl.sort_values(["ID", "TI"])

# Resetear index pra que se vea ordenado
tl_1 = tl.reset_index(drop=True)

In [143]:
# Copio para no correr denuevo lo largo de atras
tl = tl_1.copy()

In [146]:
# Ubicaciones
ubicaciones = list(tl["UBICACIÓN"].unique())
ubicaciones.sort()
# ubicaciones
tl["UBICACIÓN"].unique()

array(['WL_WL', 'Hospital_1_GA', 'WL_WL -> Hospital_1_GA',
       'Hospital_1_ICU', 'Hospital_1_ICU -> Hospital_1_SDU_WARD',
       'Hospital_1_SDU_WARD', 'PS_PS', 'WL_WL -> PS_PS', 'Hospital_1_OR',
       'Hospital_3_GA', 'Hospital_3_OR', 'WL_WL -> Hospital_3_GA',
       'Hospital_3_ICU', 'Hospital_3_SDU_WARD', 'Hospital_2_GA',
       'Hospital_2_ICU', 'WL_WL -> Hospital_2_GA',
       'Hospital_2_ICU -> Hospital_2_SDU_WARD', 'Hospital_2_SDU_WARD',
       'Hospital_3_ICU -> Hospital_3_SDU_WARD', 'Hospital_1_ED',
       'Hospital_3_ED', 'Hospital_3_ED -> Hospital_1_ED', 'Hospital_2_ED',
       'Hospital_2_ED -> Hospital_3_ED', 'Hospital_2_OR',
       'Hospital_3_ED -> Hospital_2_ED', 'Hospital_1_ED -> Hospital_3_ED',
       'Hospital_3_OR -> Hospital_3_ICU',
       'Hospital_2_ED -> Hospital_1_ED', 'Hospital_1_ED -> Hospital_2_ED',
       'Hospital_1_OR -> Hospital_1_ICU',
       'Hospital_3_OR -> Hospital_3_SDU_WARD',
       'Hospital_2_OR -> Hospital_2_ICU', 'Hospital_1_ED -> PS_PS',


Dudas
- Lo que hablamos ayer de cuando el sistema colapsaba y me pasaba del budget, eso se da porque siempre tengo que aceptar a todos los pacientes?
- Cuando llegan mas de 15 pacientes a la misma ED, los dejo entrar y los traslado instantaneamente a las otras ED?
- Si todas las ED estan llenas, que hago con los nuevos pacientes que lleguen a ED, sobre cupo nomas?, por ejemplo el hospital 1 en 92 casos tuvo mas de 15 pacientes en ED
- Generalmente el LOS en ED es cero debido a su alto costo de espera, es esta una reestricción el que no pueden tener LOS > 0 en ED?
ya que en algunos casos (246 de 80453) el LOS fue >= 12, en que casos se pudo haber dado eso?
- Entiendo que en muchos casos, debido a la regla de no mas de 400 periodos en WL, pacientes son aceptados al GA de hospitales aunque estos tengan que esperar (3827 de 10369 casos), pero porque la simulacion hospitalizo a 6542 pacientes (que no llevaban el maximo de periodos en WL), haciendolos esperar en GA cuando eso era mas caro que hacerlos esperar en WL.
- Por que en casos donde las GA no estaban llenas, existiendo 8 pacientes entre todas las GA (ciclo 4848 por ejemplo que se envian 17 pacientes de WL a PS), el modelo prefirio enviar a PS, cuando eso siempre es mas caro.

In [141]:
# Agrego costos operativos aproximados
tl["COSTO DER WL"] = 0
tl["COSTO DER ED"] = 0
tl["COSTO TRASLADO"] = 0
tl.head(20)
# dict_costo_derivar_wl[drg][unidad]
# dict_costo_derivar_ed[hospital][drg][unidad]
tl["COSTO DER WL"] = tl.apply(
    lambda x: ((p.dict_costo_derivar_wl[x["MS_GRD"]][1] + p.dict_costo_derivar_wl[x["MS_GRD"]][2] + p.dict_costo_derivar_wl[x["MS_GRD"]][3])/3)
    if x["UBICACIÓN"] == "WL_WL -> PS_PS" else 0,
    axis=1,
)

tl["COSTO DER ED"] = tl.apply(
    lambda x: ((p.dict_costo_derivar_ed[p.dict_hospitales[x["HOSPITAL"]]][x["MS_GRD"]][1] + p.dict_costo_derivar_ed[p.dict_hospitales[x["HOSPITAL"]]][x["MS_GRD"]][2] + p.dict_costo_derivar_ed[p.dict_hospitales[x["HOSPITAL"]]][x["MS_GRD"]][3])/3)
    if x["UBICACIÓN"] == f"{x['HOSPITAL']}_ED -> PS_PS" else 0,
    axis=1,
)

traslados = ['Hospital_1_ED -> Hospital_2_ED', 'Hospital_1_ED -> Hospital_3_ED', 'Hospital_2_ED -> Hospital_1_ED', 'Hospital_2_ED -> Hospital_3_ED', 'Hospital_3_ED -> Hospital_1_ED', 'Hospital_3_ED -> Hospital_2_ED']

tl["COSTO TRASLADO"] = tl.apply(
    lambda x: (p.dict_costo_traslado[1][2][x["MS_GRD"]][p.dict_unidades["SDU/WARD"]])
    if x["UBICACIÓN"] in (traslados) else 0,
    axis=1,
)

In [142]:
ciclo = 3348 #50016 #31308 #30072 #4848

display(tl[(tl["TI"] == ciclo) & ((tl["COSTO DER ED"] > 0))].head(20))
display(tl[(tl["TI"] == ciclo) & ((tl["COSTO DER WL"] > 0))].head(30))
display(tl[(tl["TI"] == ciclo) & ((tl["COSTO TRASLADO"] > 0))].head(30))

costo1 = tl[(tl["TI"] == ciclo) & ((tl["COSTO DER ED"] > 0))]["COSTO DER ED"].sum()
costo2 = tl[(tl["TI"] == ciclo) & ((tl["COSTO DER WL"] > 0))]["COSTO DER WL"].sum()
costo3 = tl[(tl["TI"] == ciclo) & ((tl["COSTO TRASLADO"] > 0))]["COSTO TRASLADO"].sum()

print(f"Total costo derivacion ED: {costo1}")
print(f"Total costo derivacion WL: {costo2}")
print(f"Total costo traslado: {costo3}")
print(f"Total costo: {costo1 + costo2 + costo3}")


,ID,MS_GRD,UBICACIÓN,TI,TF,LOS,HOSPITAL,UNIDAD,COSTO DER WL,COSTO DER ED,COSTO TRASLADO
41552,8715,3,Hospital_1_ED -> PS_PS,3348,3348,0,Hospital_1,ED,0.0,821.82583,0.0
41559,8717,3,Hospital_3_ED -> PS_PS,3348,3348,0,Hospital_3,ED,0.0,821.82583,0.0
41562,8718,3,Hospital_3_ED -> PS_PS,3348,3348,0,Hospital_3,ED,0.0,821.82583,0.0
41565,8719,3,Hospital_3_ED -> PS_PS,3348,3348,0,Hospital_3,ED,0.0,821.82583,0.0
41568,8720,3,Hospital_3_ED -> PS_PS,3348,3348,0,Hospital_3,ED,0.0,821.82583,0.0
41571,8721,3,Hospital_3_ED -> PS_PS,3348,3348,0,Hospital_3,ED,0.0,821.82583,0.0
41574,8722,3,Hospital_2_ED -> PS_PS,3348,3348,0,Hospital_2,ED,0.0,821.82583,0.0
41577,8723,3,Hospital_2_ED -> PS_PS,3348,3348,0,Hospital_2,ED,0.0,821.82583,0.0
41580,8724,3,Hospital_2_ED -> PS_PS,3348,3348,0,Hospital_2,ED,0.0,821.82583,0.0
41583,8725,3,Hospital_2_ED -> PS_PS,3348,3348,0,Hospital_2,ED,0.0,821.82583,0.0


,ID,MS_GRD,UBICACIÓN,TI,TF,LOS,HOSPITAL,UNIDAD,COSTO DER WL,COSTO DER ED,COSTO TRASLADO
36969,7762,6,WL_WL -> PS_PS,3348,3348,0,WL,WL,500.719575,0.0,0.0
37097,7789,6,WL_WL -> PS_PS,3348,3348,0,WL,WL,500.719575,0.0,0.0
37210,7811,6,WL_WL -> PS_PS,3348,3348,0,WL,WL,500.719575,0.0,0.0
37213,7812,6,WL_WL -> PS_PS,3348,3348,0,WL,WL,500.719575,0.0,0.0
37216,7813,6,WL_WL -> PS_PS,3348,3348,0,WL,WL,500.719575,0.0,0.0
37458,7860,6,WL_WL -> PS_PS,3348,3348,0,WL,WL,500.719575,0.0,0.0
37461,7861,6,WL_WL -> PS_PS,3348,3348,0,WL,WL,500.719575,0.0,0.0
37464,7862,6,WL_WL -> PS_PS,3348,3348,0,WL,WL,500.719575,0.0,0.0
37606,7891,6,WL_WL -> PS_PS,3348,3348,0,WL,WL,500.719575,0.0,0.0
37609,7892,6,WL_WL -> PS_PS,3348,3348,0,WL,WL,500.719575,0.0,0.0


,ID,MS_GRD,UBICACIÓN,TI,TF,LOS,HOSPITAL,UNIDAD,COSTO DER WL,COSTO DER ED,COSTO TRASLADO
41431,8695,1,Hospital_1_ED -> Hospital_3_ED,3348,3348,0,Hospital_1,ED,0.0,0.0,18.517418
41437,8696,1,Hospital_1_ED -> Hospital_3_ED,3348,3348,0,Hospital_1,ED,0.0,0.0,18.517418
41443,8697,1,Hospital_1_ED -> Hospital_3_ED,3348,3348,0,Hospital_1,ED,0.0,0.0,18.517418
41451,8698,1,Hospital_1_ED -> Hospital_2_ED,3348,3348,0,Hospital_1,ED,0.0,0.0,18.517418
41467,8700,1,Hospital_1_ED -> Hospital_3_ED,3348,3348,0,Hospital_1,ED,0.0,0.0,18.517418
41472,8701,1,Hospital_1_ED -> Hospital_3_ED,3348,3348,0,Hospital_1,ED,0.0,0.0,18.517418
41478,8702,1,Hospital_1_ED -> Hospital_3_ED,3348,3348,0,Hospital_1,ED,0.0,0.0,18.517418
41483,8703,1,Hospital_1_ED -> Hospital_3_ED,3348,3348,0,Hospital_1,ED,0.0,0.0,18.517418
41488,8704,1,Hospital_1_ED -> Hospital_3_ED,3348,3348,0,Hospital_1,ED,0.0,0.0,18.517418
41493,8705,2,Hospital_1_ED -> Hospital_3_ED,3348,3348,0,Hospital_1,ED,0.0,0.0,20.127646


Total costo derivacion ED: 9861.9099596
Total costo derivacion WL: 5007.195754
Total costo traslado: 275.88397659799995
Total costo: 15144.989690198


In [138]:
# Analisis cantidad de pacientes en ED
"""Se puede ver que en la gran mayoria de los casos a la ED no llegan mas de 15 pacientes, lo cual es su capacidad maxima,
en 92 casos se exedio esta capacidad maxima, en estos casos la capacidad debe ser relajanda si es que no tengo mas plata para enviarlos
al PS y tampoco los puedo atender en ningun hospital"""
# tl[(tl["UNIDAD"] == "ED") & (tl["HOSPITAL"] == "Hospital_1")].groupby("TI").size().reset_index(name = "Llegadas")["Llegadas"].value_counts().reset_index()
cantidad_llegadas = tl[(tl["UNIDAD"] == "ED") & (tl["HOSPITAL"] == "Hospital_1")].groupby("TI").size().reset_index(name = "Llegadas")
cantidad_llegadas[cantidad_llegadas["Llegadas"] > 15]

,TI,Llegadas
19,240,17
34,420,16
39,480,21
214,2592,18
275,3348,23
...,...,...
3947,47568,20
4007,48288,23
4027,48528,20
4112,49548,16


In [126]:
# Analisis LOS en ED
# tl[(tl["UNIDAD"] == "ED") & (tl["HOSPITAL"] == "Hospital_1")]["MS_GRD"].value_counts()
tl[(tl["UNIDAD"] == "ED") & (tl["LOS"] >= 12)] # 246 casos LOS mayor o igual a 12 en hospital
# tl[(tl["UNIDAD"] == "ED") & (tl["LOS"] >= 24)] # 7 casos LOS mayor o igual a 24 en hospital
# tl[(tl["UNIDAD"] == "ED")] # Esos fueron de un total de 80453 estadias en ED, lo cual es casi despreciable


,ID,MS_GRD,UBICACIÓN,TI,TF,LOS,HOSPITAL,UNIDAD,COSTO DER WL,COSTO DER ED,COSTO TRASLADO
3099,643,1,Hospital_1_ED,252,264,12,Hospital_1,ED,0.0,0.0,0.0
3103,644,1,Hospital_1_ED,252,264,12,Hospital_1,ED,0.0,0.0,0.0
3115,647,2,Hospital_3_ED,252,264,12,Hospital_3,ED,0.0,0.0,0.0
3135,652,2,Hospital_3_ED,252,264,12,Hospital_3,ED,0.0,0.0,0.0
10904,2281,1,Hospital_1_ED,852,864,12,Hospital_1,ED,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
613955,130427,1,Hospital_1_ED,50016,50028,12,Hospital_1,ED,0.0,0.0,0.0
614528,130606,1,Hospital_1_ED,50076,50088,12,Hospital_1,ED,0.0,0.0,0.0
614537,130612,2,Hospital_3_ED,50076,50088,12,Hospital_3,ED,0.0,0.0,0.0
614554,130622,2,Hospital_3_ED,50076,50088,12,Hospital_3,ED,0.0,0.0,0.0


In [9]:
# Analisis LOS en GA
tl[(tl["UNIDAD"] == "GA")]["LOS"] # 50954 casos de personas en GA
tl[(tl["UNIDAD"] == "GA") & (tl["LOS"] == 0)] # 40585 casos LOS igual a 0
tl[(tl["UNIDAD"] == "GA") & (tl["LOS"] > 0)] # 10369 casos LOS mayor a 0
ga_ids_los_positivo = tl[(tl["UNIDAD"] == "GA") & (tl["LOS"] > 0)]["ID"]
tl[(tl["ID"].isin(ga_ids_los_positivo)) & (tl["UNIDAD"] == "WL")]["LOS"].value_counts() 

LOS
4800    3827
0       1107
12       333
24       207
36       190
        ... 
2844       1
2868       1
3000       1
3072       1
2292       1
Name: count, Length: 377, dtype: int64

In [10]:
# Todos los casos a GA llegan desde la Waiting list
# Get unique IDs of patients who were in the GA unit
ga_ids = tl[tl["UNIDAD"] == "GA"]["ID"].unique()
# Filter dataframe to only include those IDs
tl[tl["ID"].isin(ga_ids)].groupby("ID").first()["UBICACIÓN"].value_counts()


UBICACIÓN
WL_WL    50954
Name: count, dtype: int64

In [11]:
ga_ids = tl[(tl["UNIDAD"] == "WL") & (tl["LOS"] == 4800)]["ID"].unique()
# Filter dataframe to only include those IDs
# tl[tl["ID"].isin(ga_ids)].groupby("ID").first()["UBICACIÓN"].value_counts()
tl[tl["ID"].isin(ga_ids)].groupby("ID").first() # 5165 personas esperaron el tiempo maximo en la WL
tl[(tl["ID"].isin(ga_ids)) & (tl["UNIDAD"] == "PS")] # 460 de estas personas fueron enviados a PS (el resto se hospitalizo)

,ID,MS_GRD,UBICACIÓN,TI,TF,LOS,HOSPITAL,UNIDAD
408,91,7,PS_PS,4848,4848,0,PS,PS
2382,567,7,PS_PS,5028,5028,0,PS,PS
3634,881,7,PS_PS,5148,5148,0,PS,PS
4138,1002,7,PS_PS,5196,5196,0,PS,PS
4592,1115,7,PS_PS,5232,5232,0,PS,PS
...,...,...,...,...,...,...,...,...
483113,117937,8,PS_PS,50028,50028,0,PS,PS
483115,117938,8,PS_PS,50028,50028,0,PS,PS
485016,118411,7,PS_PS,50196,50196,0,PS,PS
485018,118412,7,PS_PS,50196,50196,0,PS,PS


In [12]:
# Porque se mando a tanta gente a PS existiendo la posibilidad de hospitalizar?
# ga_ids = tl[(tl["UNIDAD"] == "WL") & (tl["LOS"] == 4800)]["ID"].unique()
wl_ids = tl[(tl["UNIDAD"] == "WL")]["ID"].unique()
display(len(tl[(tl["ID"].isin(wl_ids)) & (tl["UNIDAD"] == "PS") & (tl["TF"] == 4848)]))
display(len(tl[(tl["TI"] == 4848) & (tl["UNIDAD"] == "GA")].head(20)))
display(tl[(tl["TF"] == 4848) & (tl["UNIDAD"] == "GA")].head(20))


17

8

,ID,MS_GRD,UBICACIÓN,TI,TF,LOS,HOSPITAL,UNIDAD
40638,9872,5,Hospital_1_GA,4656,4848,192,Hospital_1,GA
40642,9873,5,Hospital_1_GA,4656,4848,192,Hospital_1,GA
52284,12676,6,Hospital_2_GA,4848,4848,0,Hospital_2,GA
52320,12683,8,Hospital_3_GA,4848,4848,0,Hospital_3,GA
52327,12684,8,Hospital_3_GA,4848,4848,0,Hospital_3,GA
52438,12712,6,Hospital_2_GA,4848,4848,0,Hospital_2,GA
52501,12728,8,Hospital_2_GA,4848,4848,0,Hospital_2,GA
52506,12729,8,Hospital_2_GA,4848,4848,0,Hospital_2,GA


In [13]:
# Función con que unidades a considerar

def los_case_mix_report(tl: pd.DataFrame, unidades: list = ["ICU", "OR", "SDU_WARD"]):
    # Proporción de GRDs entre todos los casos (P_j)
    variable = tl.groupby('ID')['MS_GRD'].max().reset_index()["MS_GRD"].value_counts(normalize=True).sort_index().reset_index()
    P_j = {}
    for index, row in variable.iterrows():
        P_j[int(row['MS_GRD'])] = row['proportion']

    # LOS estando ya hospitalizado promedio por GRD (A_j)
    v1 = tl[tl["UNIDAD"].isin(unidades)]
    v2 = v1.groupby('ID').agg({
        'LOS': 'sum',
        'MS_GRD': 'min'
    }).reset_index().groupby("MS_GRD")["LOS"].mean().reset_index()
    A_j = {}
    for index, row in v2.iterrows():
        A_j[int(row['MS_GRD'])] = row['LOS']

    # LOS promedio por hospital_i y GRD_j (a_i_j)
    a_i_j = {}
    for numero in range(1,4):
        v1 = tl[tl["HOSPITAL"] == f"Hospital_{numero}"]
        v2 = v1[v1["UNIDAD"].isin(unidades)]
        v3 = v2.groupby('ID').agg({
            'LOS': 'sum',
            'MS_GRD': 'min'
        }).reset_index().groupby("MS_GRD")["LOS"].mean().reset_index()
        for index, row in v3.iterrows():
            a_i_j[(numero,int(row['MS_GRD']))] = row['LOS']

    # Proporción de GRDs en cada hospital (p_i_j)
    p_i_j = {}

    for numero in range(1,4):
        # Filtro por hospital
        v1 = tl[tl["HOSPITAL"] == f"Hospital_{numero}"]
        # Filtro que este en alguna de esas unidades, cuando ya estan aceptados por el hospital
        v2 = v1[v1["UNIDAD"].isin(unidades)]
        v3 = v2.groupby('ID')['MS_GRD'].max().reset_index()["MS_GRD"].value_counts(normalize=True).sort_index().reset_index()
        for index, row in v3.iterrows():
                p_i_j[(numero,int(row['MS_GRD']))] = row['proportion']

    # LOS promedio total
    v1 = tl[tl["UNIDAD"].isin(unidades)]
    v2 = v1.groupby('ID').agg({'LOS': 'sum'}).reset_index()
    mean_los = v2["LOS"].mean()

    # Average LOS by hospital
    mean_los_i = {}
    for numero in range(1,4):
        v1 = tl[tl["HOSPITAL"] == f"Hospital_{numero}"]
        v2 = v1[v1["UNIDAD"].isin(unidades)]
        v3 = v2.groupby('ID').agg({
            'LOS': 'sum',
            'MS_GRD': 'min'
        }).reset_index()
        mean_los_i[numero] = v3["LOS"].mean()

    # Case mix adjusted LOS
    cmal_i = {}
    for i in range(1,4):
        suma = 0
        for j in range(1, 9):
            suma += a_i_j[(i, j)] * P_j[j]
        cmal_i[i] = suma

    # LOS weighted case mix
    lwcm_i = {}
    for i in range(1,4):
        suma = 0
        for j in range(1, 9):
            suma += A_j[j] * p_i_j[(i, j)]
        lwcm_i[i] = suma

    # Convertir todo a un DataFrame una sola vez
    new_rows = []
    for i in range(1,4):
        new_row = {
            'HOSPITAL': i,
            'AVERAGE LOS': mean_los_i[i],
            'CASE MIX ADJUSTED LOS': cmal_i[i],
            'LOS WEIGHTED CASE MIX': lwcm_i[i]
        }
        new_rows.append(new_row)
    new_rows_df = pd.DataFrame(new_rows)

    new_rows_df["AVERAGE LOS INDEX"] = new_rows_df["AVERAGE LOS"] / mean_los
    new_rows_df["LOS INDEX"] = new_rows_df["CASE MIX ADJUSTED LOS"] / mean_los
    new_rows_df["CASE MIX INDEX"] = new_rows_df["LOS WEIGHTED CASE MIX"] / mean_los

    new_rows_df["AVERAGE LOS DIFFER"] = new_rows_df["AVERAGE LOS"] - mean_los
    new_rows_df["DIFFER DUE TO LOS"] = new_rows_df["CASE MIX ADJUSTED LOS"] - mean_los
    new_rows_df["DIFFER DUE TO CASE MIX"] = new_rows_df["LOS WEIGHTED CASE MIX"] - mean_los

    new_rows_df["INTERACT DIFFER"] = new_rows_df["AVERAGE LOS DIFFER"] - new_rows_df["DIFFER DUE TO LOS"] - new_rows_df["DIFFER DUE TO CASE MIX"]

    return new_rows_df

# Ejemplo de uso
#unidades = ["ICU", "OR", "SDU_WARD", "GA"]
unidades = ["ICU", "OR", "SDU_WARD"]
report_df = los_case_mix_report(tl, unidades)
display(report_df)


,HOSPITAL,AVERAGE LOS,CASE MIX ADJUSTED LOS,LOS WEIGHTED CASE MIX,AVERAGE LOS INDEX,LOS INDEX,CASE MIX INDEX,AVERAGE LOS DIFFER,DIFFER DUE TO LOS,DIFFER DUE TO CASE MIX,INTERACT DIFFER
0,1,309.654573,293.038532,307.723614,1.016514,0.961968,1.010175,5.030433,-11.585609,3.099473,13.516569
1,2,296.776073,290.421790,296.849523,0.974237,0.953377,0.974478,-7.848067,-14.202351,-7.774617,14.128901
2,3,307.748436,289.821874,309.586704,1.010256,0.951408,1.016291,3.124295,-14.802266,4.962564,12.963998


In [14]:
# Matriz de transición (las valide al azar con Disco y me dio lo mismo)
matrices = {}

for hospital in range(1,4):    
    for grd in range(1, 9):
        v3 = tl[(tl["UNIDAD"].isin(["ICU", "OR", "SDU_WARD"])) & 
            (tl["MS_GRD"] == grd) & 
            (tl["HOSPITAL"] == f"Hospital_{hospital}")].sort_values(["ID", "TI"])

        transiciones = {}

        for i in range(len(v3) - 1):
            row_current = v3.iloc[i]
            row_next = v3.iloc[i + 1]

            # Verificar condiciones
            same_id = row_current['ID'] == row_next['ID']
            actual = row_current['UNIDAD']
            siguiente = row_next['UNIDAD']
            distinta = actual != siguiente
            if same_id and distinta:
                if (actual, siguiente) in transiciones:
                    transiciones[(actual, siguiente)] += 1

                else:
                    transiciones[(actual, siguiente)] = 1

            # Solo para cuando esta en SDU_WARD al ser nodo de termino
            elif not same_id and actual == "SDU_WARD":
                if (actual, actual) in transiciones:
                    transiciones[(actual, actual)] += 1
                else:
                    transiciones[(actual, actual)] = 1



        unidades = ["OR","ICU", "SDU_WARD"]
        num = len(unidades) + 1
        matriz = []
        for i in range(num):
            matriz.append([0] * num)

        for i in range(1, num):
            matriz[0][i] = unidades[i - 1]
            matriz[i][0] = unidades[i - 1]

        for i in range(1, num):
            for j in range(1, num):
                if (matriz[i][0], matriz[0][j]) in transiciones:
                    matriz[i][j] = transiciones[(matriz[i][0], matriz[0][j])]
                else:
                    matriz[i][j] = 0

        for i in range(1, num):
            total = sum(matriz[i][1:])
            for j in range(1, num):
                    matriz[i][j] = round(matriz[i][j]/total, 5)
        
        matriz[0][0] = "Unidad"
        matriz[1][1] = "0.00000"
        matriz[2][2] = "0.00000"

        matrices[(hospital, grd)] = matriz

        # print(f"Hospital {hospital}, MS_GRD: {grd}")
        # display(matriz)

In [15]:
# Generar el texto para las matrices en LaTeX
for key, m in matrices.items():
    
    texto = f"""
\\begin{{table}}[H]
    \\centering
    \\begin{{tabular}}{{lccc}}
        \\toprule
        {m[0][0]} & {m[0][1]} & {m[0][2]} & {m[0][3]} \\\\
        \\midrule
        {m[1][0]}       & {m[1][1]} & {m[1][2]} & {m[1][3]} \\\\
        {m[2][0]}      & {m[2][1]} & {m[2][2]} & {m[2][3]} \\\\
        {m[3][0]}       & {m[3][1]} & {m[3][2]} & {m[3][3]} \\\\
        \\bottomrule
    \\end{{tabular}}
    \\caption{{Matriz de transición Hospital: {key[0]} y MS\\_GRD: {key[1]}}}
    \\label{{tab:matriz_transicion_Hospital{key[0]}_GRD{key[1]}}}
\\end{{table}}
    """

    texto = texto.replace("SDU_WARD", "SDU\\_WARD")
    # print(texto)